In [0]:
import pandas as pd

In [0]:
year = '2014'
matchs = pd.read_csv('/content/gdrive/My Drive/Data Science/Cartola/2014/2014_partidas.csv')
scouts = pd.read_csv('/content/gdrive/My Drive/Data Science/Cartola/Standardized/scouts_2014.csv')
team_dict = pd.read_csv('/content/gdrive/My Drive/Data Science/Cartola/times_ids.csv')

In [0]:
def add_column_name(id):
  name = team_dict[team_dict['id'] == id]['nome.cartola'].values[0]
  return name


def replace_slug_by_id(slug):
  name = team_dict[team_dict['nome.cbf'] == slug]['id'].values
  return name[0]


def match_result(team, round):
  match = matchs[((matchs['home_team'] == team) | 
                  (matchs['away_team'] == team)) & 
                 (matchs['round'] == round)]['score'].values[0]
  
  if len(match) < 5:
    return 0
  
  match = match.split('x')
  
  is_home = matchs[(matchs['home_team'] == team) & (matchs['round'] == round)].shape[0]
  
  if is_home == 1:
    if int(match[0]) > int(match[1]):
      return 3
    
    elif int(match[0]) < int(match[1]):
      return 0
    
    return 1
    
  if int(match[0]) > int(match[1]):
    return 0

  elif int(match[0]) < int(match[1]):
    return 3

  return 1


def cs_per_round(row, data):
  team = row['team']
  round = row['round']
  
  data = data[data['team'] == team]
  data = data[data['round'] <= round]
  
  score = data['result_match'].sum()
  
  return score


def is_home_team(team, round):
  is_home = matchs[(matchs['home_team'] == team) & 
                   (matchs['round'] == round)].shape[0]
  return is_home


def goals_conceded(team, round): # only current GKs
  
  data = scouts[scouts['rodada'] == round]
  
  team = data[data['clube_id'] == team]
  
  return team['GS'].sum()


def goals_scored(team, round): # only current players
  
  data = scouts[scouts['rodada'] == round]
  
  team = data[data['clube_id'] == team]
  
  return team['G'].sum()


def wcg	(team, round): # only current GKs
  
  data = scouts[scouts['rodada'] == round]
  
  team = data[(data['clube_id'] == team) 
             & (data['posicao'] == 'gol')]
  
  return team['SG'].sum()


def ag_scored(team, round):
  
  goals = goals_scored(team, round)
  avg = goals/round
  
  return avg


def ag_conceded(team, round):
  
  goals = goals_conceded(team, round)
  avg = goals/round
  
  return avg

In [0]:
matchs['home_team'] = matchs.apply(lambda  row: replace_slug_by_id(row['home_team']), axis=1)
matchs['away_team'] = matchs.apply(lambda  row: replace_slug_by_id(row['away_team']), axis=1)

team_names = matchs['home_team'].unique()
tr = pd.DataFrame({'team': [], 'round': []})

for i in range(1, 39):
    df = pd.DataFrame({'team': team_names,
                       'round': [i] * 20})
    tr = tr.append(df, sort=False)

    
# sem dados para a última rodada do campeonato de 2015
if year == '2015':
  tr = tr[:-20]
    
tr['round'] = tr['round'].astype(int)

In [0]:
tr['result_match'] = tr.apply(lambda row: match_result(row['team'], row['round']), axis=1)
tr['championship_score'] = tr.apply(lambda row: cs_per_round(row, tr), axis=1)
tr['home_team'] = tr.apply(lambda row: is_home_team(row['team'], row['round']), axis=1)
tr['goals_conceded'] = tr.apply(lambda row: goals_conceded(row['team'], row['round']), axis=1)
tr['goals_scored'] = tr.apply(lambda row: goals_scored(row['team'], row['round']), axis=1)
tr['ag_scored'] = tr.apply(lambda row: ag_scored(row['team'], row['round']), axis=1) # goal average scored
tr['ag_conceded'] = tr.apply(lambda row: ag_conceded(row['team'], row['round']), axis=1) # goal average conceded
tr['wcg'] = tr.apply(lambda row: wcg(row['team'], row['round']), axis=1) # without conceding a goal
tr['clube_nome'] = tr.apply(lambda row: add_column_name(row['team']), axis=1)

In [283]:
tr.tail(20)

,team,round,result_match,championship_score,home_team,goals_conceded,goals_scored,ag_scored,ag_conceded,wcg,clube_nome
0,262.0,38,1,52,0,47,46,1.210526,1.236842,12,Flamengo
1,266.0,38,0,61,0,42,57,1.500000,1.105263,14,Fluminense
2,276.0,38,0,70,0,40,56,1.473684,1.052632,13,São Paulo
3,277.0,38,3,53,0,35,41,1.078947,0.921053,15,Santos
4,293.0,38,1,54,0,42,42,1.105263,1.105263,12,Atlético-PR
5,282.0,38,1,62,0,39,45,1.184211,1.026316,14,Atlético-MG
6,265.0,38,0,37,0,42,28,0.736842,1.105263,10,Bahia
7,285.0,38,3,69,0,40,51,1.342105,1.052632,16,Internacional
8,206.0,38,0,32,0,56,28,0.736842,1.473684,11,Criciuma
9,315.0,38,0,43,0,39,34,0.894737,1.026316,12,Chapecoense


In [0]:
tr.to_csv('/content/gdrive/My Drive/Data Science/Cartola/TeamRound/tr_2014.csv', index=False)